## Transferring Files to and from an S3 Bucket

Retrieve files from an S3 bucket before executing a task, and upload files to an S3 bucket after a task, using the `boto3` library.

### Procedure

Transfer a file from an S3 bucket to the local filesystem using the boto3 library. 

In the following example a zip file is downloaded from an S3 bucket before electron execution. The electron processes the file's contents, then the processed files are uploaded back to the S3 bucket.

1. Define two Covalent `FileTransfer` objects and a Covalent `S3` strategy object:

In [ ]:
import covalent as ct
from typing import List
from pathlib import Path
from skimage import io, color

strategy = ct.fs_strategies.S3()

unprocessed_filepath = "unprocessed_file.png"
processed_filepath = "processed_file.png"

s3_source_path = f"s3://covalent-tmp/{unprocessed_filepath}"
s3_dest_path = f"s3://covalent-tmp/{processed_filepath}"

ft_1 = ct.fs.FileTransfer(s3_source_path, unprocessed_filepath, strategy=strategy)
ft_2 = ct.fs.FileTransfer(processed_filepath, s3_dest_path, strategy=strategy, order=ct.fs.Order.AFTER)


2. Define an electron to:
    1. Download the unprocessed file from S3
    2. Perform some processing on the contents
    3. Upload the processed file to S3

In [ ]:
@ct.electron(files=[ft_1, ft_2]) # ft_1 is done before the electron is executed and ft_2 after.
def to_grayscale(files: List[ct.fs.FileTransfer] = None):
    
    # Get the downloaded file's path
    image_path = files[0].to_file
    
    # Convert the image to grayscale
    img = io.imread(image_path)
    gray_img = color.rgb2gray(img)

    # Save the grayscale image to the to-be-uploaded file's path
    io.imsave(files[1].from_file, gray_img)


3. Create and dispatch a lattice to run the electron:

In [ ]:
@ct.lattice
def process_s3_data():
    return to_grayscale()

dispatch_id = ct.dispatch(process_s3_data)()

Notes:
- This example illustrates a typical pattern in which files are downloaded from remote storage, are processed, and the results are uploaded to the same remote storage. Other scenarios can of course be implemented with the Covalent components illustrated here (`FileTransfer`, `FileTransferStrategy`, `@electron`).
- The example puts everything in one electron (file download, processing, file upload). For a real-world scenario of any complexity, a better practice would be to break the task into small sub-tasks, each in its own electron.

### See Also

[Transferring Local Files During Workflows](./file_transfers_for_workflows_local.ipynb)

[Transferring Remote Files During Workflows](./file_transfers_to_remote.ipynb)